# Capitulo 02 Instalacao Configuracao Httpfs

Notebook gerado automaticamente a partir do código fonte python.


In [ ]:
# -*- coding: utf-8 -*-
"""
capitulo_02_instalacao_configuracao_httpfs
"""

# capitulo_02_instalacao_configuracao_httpfs
import duckdb
import os

# Exemplo/Bloco 1
import duckdb
con = duckdb.connect(database=':memory:')

con.execute("""
INSTALL httpfs;
LOAD httpfs;
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Conecte-se ao DuckDB e execute:
D INSTALL httpfs;
D LOAD httpfs;

-- Mensagem de confirmação:
-- Extension httpfs loaded successfully
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Sem instalar explicitamente, apenas use:
SELECT * FROM 'https://example.com/data.parquet';

-- DuckDB automaticamente:
-- 1. Detecta que httpfs é necessário
-- 2. Baixa e instala a extensão
-- 3. Carrega a extensão
-- 4. Executa a query
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Ver todas as extensões instaladas
SELECT * FROM duckdb_extensions();
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Teste básico de conectividade
SELECT count(*) FROM 'https://shell.duckdb.org/data/tpch/0_01/parquet/lineitem.parquet';
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Configurar proxy via SET
SET http_proxy = 'http://proxy.company.com:8080';
SET http_proxy_username = 'your_username';
SET http_proxy_password = 'your_password';
""")
print(con.fetchall()) # Inspect result

con.execute("""
CREATE SECRET http_proxy (
    TYPE http,
    HTTP_PROXY 'http://proxy.company.com:8080',
    HTTP_PROXY_USERNAME 'your_username',
    HTTP_PROXY_PASSWORD 'your_password'
);
""")
print(con.fetchall()) # Inspect result

con.execute("""
LOAD httpfs;
SET ca_cert_file = '/path/to/certificate.pem';
SET enable_server_cert_verification = true;
""")
print(con.fetchall()) # Inspect result

con.execute("""
SET enable_server_cert_verification = false;
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Dataset público do DuckDB
SELECT *
FROM 'https://shell.duckdb.org/data/tpch/0_01/parquet/lineitem.parquet'
LIMIT 10;
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Arquivo CSV público
SELECT *
FROM read_csv_auto('https://example.com/data.csv')
LIMIT 5;
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Ver estrutura sem baixar todo o arquivo
SELECT *
FROM parquet_schema('https://shell.duckdb.org/data/tpch/0_01/parquet/customer.parquet');
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Lê apenas a coluna 'column_a' (não baixa o arquivo inteiro)
SELECT column_a
FROM 'https://example.com/large_file.parquet';
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Lê apenas metadados para contar linhas
SELECT count(*)
FROM 'https://shell.duckdb.org/data/tpch/0_01/parquet/lineitem.parquet';

-- Retorna resultado quase instantaneamente, sem baixar 60+ MB de dados
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Este comando provavelmente baixará o arquivo completo
SELECT column1
FROM read_csv_auto('https://example.com/large.csv');
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Definir timeout para requisições HTTP (em ms)
SET http_timeout = 30000; -- 30 segundos
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Controlar paralelismo de downloads
SET threads = 4;
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Reutilizar conexões HTTP para múltiplas requisições
SET http_keep_alive = true;
""")
print(con.fetchall()) # Inspect result

con.execute("""
CREATE SECRET http_auth (
    TYPE http,
    BEARER_TOKEN 'your_bearer_token_here'
);
""")
print(con.fetchall()) # Inspect result

con.execute("""
CREATE SECRET http_auth (
    TYPE http,
    EXTRA_HTTP_HEADERS MAP {
        'Authorization': 'Bearer token_value',
        'X-Custom-Header': 'custom_value'
    }
);
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Instalar explicitamente
INSTALL httpfs;
LOAD httpfs;
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Configurar proxy se necessário
SET http_proxy = 'http://your-proxy:port';
""")
print(con.fetchall()) # Inspect result

con.execute("""
SET enable_server_cert_verification = false;
""")
print(con.fetchall()) # Inspect result

con.execute("""
SET ca_cert_file = '/path/to/your/ca-bundle.crt';
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Aumentar timeout
SET http_timeout = 60000; -- 60 segundos
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- 1. Instale a extensão httpfs
INSTALL httpfs;
LOAD httpfs;

-- 2. Verifique se foi instalada corretamente
SELECT * FROM duckdb_extensions() WHERE name = 'httpfs';
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Leia um arquivo Parquet público do DuckDB
SELECT *
FROM 'https://shell.duckdb.org/data/tpch/0_01/parquet/nation.parquet';
""")
print(con.fetchall()) # Inspect result

con.execute("""
-- Compare o tempo de execução
-- Query 1: Count (rápido - só metadados)
SELECT count(*)
FROM 'https://shell.duckdb.org/data/tpch/0_01/parquet/lineitem.parquet';

-- Query 2: Select * (mais lento - download completo)
SELECT *
FROM 'https://shell.duckdb.org/data/tpch/0_01/parquet/lineitem.parquet'
LIMIT 10;
""")
print(con.fetchall()) # Inspect result


